In [116]:
import requests
import json
import pandas as pd

URL = 'http://search:8983/solr/news/select'
search_words = "문재인"
num_row = 100
total = 1
result_docs = []


In [129]:
start = 0
while total > start :
    params = {'debugQuery':'on','indent': 'on', 'q': 'title:"(' + search_words  + ')"~100  AND date:2017-03-29', 'sort': 'datetime desc', 'wt': 'json', 'start':start, 'rows':num_row}
    res = requests.get(URL, params=params)
    response = json.loads(res.text)
    total = response["response"]["numFound"]
    docs = response["response"]["docs"]
    #print("total:%d, start at %d" %(total, start))
    start = start + num_row
    result_docs.append(docs)
    

In [130]:
merged_result = reduce(lambda x,y: x+y,result_docs)
#for doc in merged_result:
#    print(doc['aid'] + ":" +doc['title'])

In [131]:
from pandas.io.json import json_normalize
df_result = json_normalize(merged_result)

In [132]:
df_wo_mt=df_result[df_result['oid']!='008']

In [133]:
df_wo_mt['title']=df_wo_mt['title'].str.replace(r'\[[A-Za-z가-힣0-9\= ]+\]|\([A-Za-z가-힣0-9\= ]+\)|\<[A-Za-z가-힣0-9\= ]+\>','')

/opt/conda/lib/python3.5/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [134]:
%%time
from konlpy.tag import Mecab
mecab = Mecab()
pos = lambda d: ['/'.join(p) for p in mecab.pos(d)]
def mecab_words(sentence):
    temp = sentence.split('▶')
    if(len(temp)>1):sentence = temp[0]

    temp = sentence.split('☞')
    if(len(temp)>1):sentence = temp[0]

    temp = sentence.split('ⓒ')
    if(len(temp)>1):sentence = temp[0]
        
        
    #word_list = pos(sentence)
    word_list = mecab.nouns(sentence)
    #return " ".join(word_list)
    return " ".join(filter(lambda x: len(x.strip())>1, word_list))
    #return filter(lambda x: len(x)>1, " ".join(word_list))

df_wo_mt['title_me_noun'] = df_wo_mt['title'].apply(mecab_words)

CPU times: user 584 ms, sys: 4 ms, total: 588 ms
Wall time: 595 ms


/opt/conda/lib/python3.5/site-packages/ipykernel/__main__.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [135]:
%%time
from collections import Counter

r1 = df_wo_mt['title_me_noun'].tolist()
result = Counter(" ".join(r1).split(" "))


CPU times: user 12 ms, sys: 0 ns, total: 12 ms
Wall time: 11.5 ms


In [136]:
result.most_common(1000)

[('문재인', 5626),
 ('안희정', 1025),
 ('안철수', 447),
 ('이재명', 430),
 ('대선', 397),
 ('반기문', 366),
 ('대표', 305),
 ('국민', 285),
 ('호남', 280),
 ('민주당', 274),
 ('대통령', 265),
 ('문재', 258),
 ('후보', 258),
 ('경선', 257),
 ('정권', 241),
 ('대세', 224),
 ('캠프', 216),
 ('교체', 189),
 ('발언', 163),
 ('일자리', 163),
 ('비판', 151),
 ('사드', 150),
 ('황교안', 147),
 ('정치', 146),
 ('박지원', 143),
 ('영입', 135),
 ('지지율', 128),
 ('정책', 124),
 ('탄핵', 124),
 ('공약', 122),
 ('방문', 119),
 ('지지', 119),
 ('유승민', 118),
 ('박근혜', 116),
 ('주자', 110),
 ('홍준표', 104),
 ('정부', 102),
 ('논란', 98),
 ('안보', 97),
 ('촛불', 95),
 ('개헌', 93),
 ('발표', 93),
 ('인사', 93),
 ('토론', 93),
 ('행보', 92),
 ('민주', 91),
 ('선언', 90),
 ('통합', 89),
 ('박원순', 88),
 ('경제', 87),
 ('출마', 85),
 ('압승', 84),
 ('청산', 80),
 ('특검', 80),
 ('정당', 78),
 ('대결', 78),
 ('보수', 78),
 ('충청', 77),
 ('적폐', 76),
 ('개혁', 74),
 ('세력', 74),
 ('연장', 73),
 ('남경필', 72),
 ('노무현', 72),
 ('합류', 70),
 ('참석', 70),
 ('사람', 69),
 ('민심', 69),
 ('가능', 68),
 ('준비', 67),
 ('의혹', 67),
 ('한국', 66),
 ('전인범',